In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HUGGINGFACE_TOKEN")
login(token=hf_token)
HUB_MODEL_ID = "s0hell/nllb-jesc-lora"

In [1]:
!pip install -q transformers datasets peft accelerate sentencepiece sacrebleu evaluate bitsandbytes
!pip install -qU transformers

import os
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq
)
from peft import LoraConfig, get_peft_model, TaskType, PeftModel
import numpy as np
import evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 26.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 36.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/

2025-11-11 10:20:51.956846: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762856452.165591      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762856452.223739      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [2]:
class Config:
    # Model
    model_name = "facebook/nllb-200-distilled-600M"

    # LoRA parameters
    lora_r = 16  # Rank
    lora_alpha = 32  # Scaling factor
    lora_dropout = 0.1
    target_modules = ["q_proj", "v_proj"]  # Apply LoRA to attention layers

    # Training parameters - SAVES TO KAGGLE WORKING DIRECTORY
    output_dir = "/kaggle/working/nllb-jesc-lora"
    num_epochs = 3
    batch_size = 4
    gradient_accumulation_steps = 4
    learning_rate = 2e-4
    warmup_steps = 500
    max_length = 128

    # Languages (NLLB-200 language codes)
    src_lang = "jpn_Jpan"  # Japanese
    tgt_lang = "eng_Latn"  # English

    # Checkpoint settings
    save_steps = 500
    save_total_limit = 3

    # Resume from checkpoint (set this to the checkpoint folder path if resuming)
    # If you downloaded checkpoints from a previous run, extract them to /kaggle/working/
    resume_from_checkpoint = None  # e.g., "/kaggle/working/nllb-jesc-lora/checkpoint-1000"

config = Config()

# Create output directory if it doesn't exist
os.makedirs(config.output_dir, exist_ok=True)
print(f"✓ Checkpoints will be saved to: {config.output_dir}")
print(f"✓ Download checkpoints from 'Output' tab after each session")

✓ Checkpoints will be saved to: /kaggle/working/nllb-jesc-lora
✓ Download checkpoints from 'Output' tab after each session


In [3]:
print("Loading JESC dataset...")
dataset = load_dataset("nntsuzu/JESC")

# Split into train/validation (JESC only has train split)
dataset = dataset["train"].train_test_split(test_size=0.01, seed=42)
train_dataset = dataset["train"]
eval_dataset = dataset["test"]

print(f"Train size: {len(train_dataset)}")
print(f"Eval size: {len(eval_dataset)}")
print(f"Sample: {train_dataset[0]}")


Loading JESC dataset...


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/176M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2801388 [00:00<?, ? examples/s]

Train size: 2773374
Eval size: 28014
Sample: {'translation': {'en': "i'm still an apprentice.", 'ja': '私はまだ見習い。'}}


In [4]:
print(f"Loading model: {config.model_name}")
tokenizer = AutoTokenizer.from_pretrained(config.model_name)

# Set source and target languages
tokenizer.src_lang = config.src_lang
tokenizer.tgt_lang = config.tgt_lang

# Load model
model = AutoModelForSeq2SeqLM.from_pretrained(
    config.model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

print(f"Model loaded. Parameters: {model.num_parameters():,}")

Loading model: facebook/nllb-200-distilled-600M


tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

...

`torch_dtype` is deprecated! Use `dtype` instead!


pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

Model loaded. Parameters: 615,073,792


In [5]:
lora_config = LoraConfig(
    r=config.lora_r,
    lora_alpha=config.lora_alpha,
    target_modules=config.target_modules,
    lora_dropout=config.lora_dropout,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

# Apply LoRA
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 2,359,296 || all params: 617,433,088 || trainable%: 0.3821


In [6]:
def preprocess_function(examples):
    """Tokenize Japanese-English pairs"""
    tokenizer.src_lang = config.src_lang

    inputs = [ex["ja"] for ex in examples["translation"]]
    targets = [ex["en"] for ex in examples["translation"]]

    # Tokenize inputs
    model_inputs = tokenizer(
        inputs,
        max_length=config.max_length,
        truncation=True,
        padding=False
    )

    # Tokenize targets
    tokenizer.tgt_lang = config.tgt_lang
    labels = tokenizer(
        targets,
        max_length=config.max_length,
        truncation=True,
        padding=False
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


print("Preprocessing datasets...")
tokenized_train = train_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=train_dataset.column_names
)
tokenized_eval = eval_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=eval_dataset.column_names
)

Preprocessing datasets...


Map:   0%|          | 0/2773374 [00:00<?, ? examples/s]

Map:   0%|          | 0/28014 [00:00<?, ? examples/s]

In [7]:
metric = evaluate.load("sacrebleu")

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # Replace -100 in labels (used for padding)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    # Decode predictions and labels
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Compute BLEU score
    result = metric.compute(predictions=decoded_preds, references=[[label] for label in decoded_labels])

    return {"bleu": result["score"]}

In [13]:
training_args = Seq2SeqTrainingArguments(
    output_dir=config.output_dir,
    num_train_epochs=config.num_epochs,
    per_device_train_batch_size=config.batch_size,
    per_device_eval_batch_size=config.batch_size,
    gradient_accumulation_steps=config.gradient_accumulation_steps,
    learning_rate=config.learning_rate,
    warmup_steps=config.warmup_steps,
    weight_decay=0.01,
    logging_dir=f"{config.output_dir}/logs",
    logging_steps=100,
    save_steps=1000  # More frequent
    eval_strategy="epoch",  # Only evaluate at end of each epoch
    save_strategy="steps"  # Changed back to steps
    save_total_limit=2,
    predict_with_generate=True,
    push_to_hub=True  # ENABLED
    hub_strategy="every_save"  # NEW
    hub_token=hf_token  # NEW
    generation_max_length=config.max_length,
    fp16=True,
    load_best_model_at_end=False  # Can't use with hub
    metric_for_best_model="bleu",
    greater_is_better=True,
    report_to="none",
    # Important for resuming
    save_safetensors=True,
)

In [15]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

The model is already on multiple devices. Skipping the move to device specified in `args`.


In [16]:
print("Starting training...")
print(f"Resume from checkpoint: {config.resume_from_checkpoint}")

trainer.train(resume_from_checkpoint=config.resume_from_checkpoint)

Starting training...
Resume from checkpoint: None


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
print("Saving final LoRA adapter...")
model.save_pretrained(f"{config.output_dir}/final_adapter")
tokenizer.save_pretrained(f"{config.output_dir}/final_adapter")

print("Training complete!")
print("=" * 80)
print("IMPORTANT: Download your model from the 'Output' tab on the right!")
print("Click 'Save Version' and download the output files.")
print("=" * 80)

In [ ]:
def translate(text, src_lang="jpn_Jpan", tgt_lang="eng_Latn"):
    """Translate text using the fine-tuned model"""
    tokenizer.src_lang = src_lang
    tokenizer.tgt_lang = tgt_lang

    inputs = tokenizer(text, return_tensors="pt", max_length=config.max_length, truncation=True)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        generated_tokens = model.generate(
            **inputs,
            forced_bos_token_id=tokenizer.convert_tokens_to_ids(tgt_lang),
            max_length=config.max_length,
            num_beams=5,
            early_stopping=True
        )

    translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    return translation

# Test translation
test_sentence = "こんにちは、元気ですか？"
translation = translate(test_sentence)
print(f"Japanese: {test_sentence}")
print(f"English: {translation}")

# ============================================================================
# HOW TO RESUME TRAINING IN A NEW KAGGLE SESSION
# ============================================================================

"""
TO RESUME TRAINING IN A NEW KAGGLE SESSION:

1. Create a new notebook version
2. In the 'Input' section (right sidebar), add your previous output as a dataset:
   - Click "+ Add Input"
   - Select "Your Work" > "Datasets"
   - Find and add your previous notebook's output
   
3. Upload the checkpoint files to /kaggle/working/:
   - The checkpoints will be in /kaggle/input/your-previous-run/nllb-jesc-lora/
   - Copy them: 
     !cp -r /kaggle/input/your-previous-run/nllb-jesc-lora /kaggle/working/
   
4. Set the checkpoint path in Config:
   config.resume_from_checkpoint = "/kaggle/working/nllb-jesc-lora/checkpoint-1000"
   
5. Run the notebook - it will continue from that checkpoint

ALTERNATIVE: Let it run to completion in one session (recommended)
- Kaggle gives 30 hours/week of GPU time with P100 or T4
- This training should complete in 8-12 hours
- Enable "Internet" in settings (right sidebar) to download model/dataset

TO LOAD THE FINAL MODEL FOR INFERENCE (in a new notebook):

# Add your training output as input data source

# Load base model
model = AutoModelForSeq2SeqLM.from_pretrained(
    "facebook/nllb-200-distilled-600M",
    torch_dtype=torch.float16,
    device_map="auto"
)

# Load LoRA adapter
model = PeftModel.from_pretrained(
    model, 
    "/kaggle/input/your-training-output/nllb-jesc-lora/final_adapter"
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    "/kaggle/input/your-training-output/nllb-jesc-lora/final_adapter"
)

# Use the translate() function

KAGGLE TIPS:
- Enable GPU: Settings (right) > Accelerator > GPU T4 x2 or P100
- Enable Internet: Settings > Internet > On (required for downloading models)
- Session limit: 12 hours continuous, 30 hours/week GPU quota
- Save your work: Click "Save Version" regularly
- /kaggle/working/ is your output directory - download after training
- Dataset is cached automatically after first download
"""